In [ ]:
import torch 
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2") 
model = AutoModelForCausalLM.from_pretrained("skt/kogpt2-base-v2") 

In [ ]:
from datasets import load_dataset 

split_dict = {
    "train": "train[:8000]",
    "test": "train[8000:10000]",
    "unused": "train[10000:]",
}
dataset = load_dataset("heegyu/kowikitext", split=split_dict)
del dataset["unused"]
dataset

In [ ]:
tokenized_dataset = dataset.map(
    lambda batch: tokenizer([f"{ti}\n{te}" for ti, te in zip(batch["title"], batch["text"])]),
    batched=True,
    num_proc=2,
    remove_columns=dataset['train'].column_names,
)
tokenized_dataset

In [ ]:
max_length = 512
def group_texts(batched_sample):
    sample = {k: v[0] for k, v in batched_sample.items()}

    if sample['input_ids'][-1] != tokenizer.eos_token_id:
        for k in sample.keys():
            sample[k].append(
                tokenizer.eos_token_id if k == "input_ids" else sample[k][-1]
            )

    result = {k: [v[i: i + max_length] for i in range(0, len(v), max_length)] for k, v in sample.items()}
    return result

grouped_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1,
    num_proc=2,
)
print(len(grouped_dataset['train'][0]['input_ids']))
print(grouped_dataset)

In [ ]:
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
collator([grouped_dataset['train'][i] for i in range(1)])